In [2]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests

image = Image.open('e13b/reallife_cgi_1212991940-0.tif').convert("RGB")

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'003180049: 00238-0145* 1172 /0000120000/'

In [1]:
import pathlib

# ⑆ (transit: used to delimit a bank code);
# ⑈ (on-us: used to delimit a customer account number);
# ⑇ (amount: used to delimit a transaction amount);
# ⑉ (dash: used to delimit parts of numbers—e.g., routing numbers or account numbers).
char_map = {"A": "⑆", "B": "⑇", "C": "⑈", "D": "⑉"}

# convert dataset to EasyOCR format

images = sorted(pathlib.Path("e13b").rglob("*.tif"), key=lambda x: x.stem)
labels = sorted(pathlib.Path("e13b").rglob("*.gt.txt"), key=lambda x: x.stem)

True

In [8]:
import pandas as pd

data = []
for image_path, label_path in zip(images, labels):
    label = label_path.read_text().strip()
    for key, value in char_map.items():
        label = label.replace(key, value)
    data.append({
        "filename": image_path.name,
        "words": label
    })

df = pd.DataFrame(data)

In [10]:
df.head()
df.to_csv("labels.csv", index=False)

In [11]:
# convert all tifs to jpgs
import cv2
import numpy as np

for image_path in images:
    image = cv2.imread(str(image_path), cv2.IMREAD_UNCHANGED)
    cv2.imwrite(f"o/data/{image_path.stem}.jpg", image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])